# 使用DQN训练Atari Donkey Kong

本notebook实现了一个DQN代理来玩Atari游戏Donkey Kong，并包含以下特性：
- 并行训练多个游戏环境
- 预处理游戏帧以提高训练效率
- 使用优先经验回放提高训练质量
- 训练日志记录
- 定期保存模型
- 定期评估并录制游戏视频

## 1. 安装必要的依赖

In [37]:
# 安装必要的库
# %pip install stable-baselines3[extra] gymnasium[atari] numpy matplotlib opencv-python tensorboard autorom[accept-rom-license]

## 2. 导入库

In [38]:
import os
import random
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from gymnasium.wrappers import RecordVideo, FrameStackObservation
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from collections import deque, namedtuple
from tqdm.notebook import tqdm
import cv2
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.atari_wrappers import AtariWrapper
import ale_py
from gymnasium import spaces

# 设置随机种子，保证实验可复现
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

## 3. 配置参数

In [39]:
# 环境参数
ENV_NAME = "ALE/DonkeyKong-v5"
NUM_ENVS = 3  # 并行环境数量
FRAME_SKIP = 4  # 跳帧数，每隔4帧进行一次决策

# 模型参数
BATCH_SIZE = 32
GAMMA = 0.99  # 折扣因子
LEARNING_RATE = 0.0001
MEMORY_SIZE = 100000  # 经验回放缓冲区大小
TARGET_UPDATE = 10000  # 目标网络更新频率

# 训练参数
NUM_FRAMES = 10_000_000  # 总训练帧数
NUM_FRAMES = 1000  # 总训练帧数
EPSILON_START = 1.0
EPSILON_END = 0.1
EPSILON_DECAY = 5_000_000

# 保存和评估参数
SAVE_INTERVAL = 100_000  # 保存模型的间隔（帧数）
SAVE_INTERVAL = 250  # 保存模型的间隔（帧数）
EVAL_INTERVAL = 50_000   # 评估模型的间隔（帧数）
EVAL_INTERVAL = 250   # 评估模型的间隔（帧数）
EVAL_EPISODES = 3       # 每次评估的游戏局数

# 创建保存模型和日志的目录
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
SAVE_PATH = f"./models/donkey_kong_{timestamp}"
LOG_PATH = f"./logs/donkey_kong_{timestamp}"
VIDEO_PATH = f"./videos/donkey_kong_{timestamp}"

for path in [SAVE_PATH, LOG_PATH, VIDEO_PATH]:
    if not os.path.exists(path):
        os.makedirs(path)

# 设置设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


## 4. 环境预处理

In [40]:
# 限制动作空间,减少 agent 的无用动作
class ActionRestrictWrapper(gym.ActionWrapper):
    def __init__(self, env, allowed_actions):
        super().__init__(env)
        self.allowed_actions = allowed_actions
        self.action_space = spaces.Discrete(len(self.allowed_actions))

    def action(self, act):
        # 把 agent 输出的动作索引映射成原动作编号
        return self.allowed_actions[act]

    def reverse_action(self, act):
        return self.allowed_actions.index(act)

# 创建预处理后的环境的函数
def make_env(env_id, idx, capture_video=False, run_name=None):
    def thunk():
        import ale_py
        
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = RecordVideo(
                env,
                VIDEO_PATH,
                episode_trigger=lambda x: True,
                name_prefix=f"donkey_kong_{run_name}"
            )
        else:
            env = gym.make(env_id)

        useful_actions = [0,1,2,3,4,6,7]
        env = ActionRestrictWrapper(env, useful_actions)
        env = AtariWrapper(env, terminal_on_life_loss=True, frame_skip=FRAME_SKIP)
        env = FrameStackObservation(env, 4)  # 堆叠4帧以捕获时间信息
        return env
    return thunk

# 创建并行环境
def make_vec_env(env_id, num_envs, seed=SEED):
    env_fns = [make_env(env_id, i) for i in range(num_envs)]
    envs = SubprocVecEnv(env_fns)
    envs.seed(seed)
    return envs

## 5. DQN网络模型

In [41]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    
    def forward(self, x):
        # 输入形状: (batch, stack_frames, height, width)
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

## 6. 优先经验回放

In [42]:
# 使用优先经验回放提高训练效率
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6, beta_start=0.4, beta_frames=100000):
        self.capacity = capacity
        self.alpha = alpha  # 控制优先级的程度
        self.beta_start = beta_start
        self.beta_frames = beta_frames
        self.frame = 1  # 当前帧，用于beta计算
        self.buffer = []
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        self.position = 0
        self.Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))
    
    def beta_by_frame(self, frame_idx):
        # beta从beta_start线性增加到1.0
        return min(1.0, self.beta_start + frame_idx * (1.0 - self.beta_start) / self.beta_frames)
    
    def push(self, *args):
        # 添加新的经验
        max_prio = np.max(self.priorities) if self.buffer else 1.0
        
        if len(self.buffer) < self.capacity:
            self.buffer.append(self.Transition(*args))
        else:
            self.buffer[self.position] = self.Transition(*args)
        
        self.priorities[self.position] = max_prio
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.position]
        
        # 计算采样概率
        probs = prios ** self.alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]
        
        # 计算重要性采样权重
        beta = self.beta_by_frame(self.frame)
        self.frame += 1
        
        weights = (len(self.buffer) * probs[indices]) ** (-beta)
        weights /= weights.max()
        weights = torch.tensor(weights, device=device, dtype=torch.float32)
        
        # 转换为批量处理格式
        batch = self.Transition(*zip(*samples))
        states = torch.cat(batch.state)
        actions = torch.tensor(batch.action, device=device)
        rewards = torch.tensor(batch.reward, device=device, dtype=torch.float32)
        next_states = torch.cat(batch.next_state)
        dones = torch.tensor(batch.done, device=device, dtype=torch.float32)
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, indices, priorities):
        # 更新优先级
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority
    
    def __len__(self):
        return len(self.buffer)

## 7. DQN代理

In [43]:
class DQNAgent:
    def __init__(self, state_shape, n_actions):
        self.state_shape = state_shape
        self.n_actions = n_actions
        
        # 创建策略网络和目标网络
        self.policy_net = DQN(state_shape, n_actions).to(device)
        self.target_net = DQN(state_shape, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()  # 目标网络不需要计算梯度
        
        # 设置优化器
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LEARNING_RATE)
        
        # 创建经验回放缓冲区
        self.memory = PrioritizedReplayBuffer(MEMORY_SIZE)
        
        # 训练相关参数
        self.steps_done = 0
        self.epsilon = EPSILON_START
        
        # 日志记录器
        self.writer = SummaryWriter(LOG_PATH)
    
    def select_action(self, state, eval_mode=False):
        # ε-贪婪策略选择动作
        sample = random.random()
        # 在评估模式下，始终选择最佳动作
        if eval_mode:
            eps_threshold = 0.05  # 评估时使用小的epsilon，增加一些探索性
        else:
            # 线性衰减epsilon
            self.epsilon = max(EPSILON_END, EPSILON_START - self.steps_done / EPSILON_DECAY)
            eps_threshold = self.epsilon
            
        if sample > eps_threshold:
            with torch.no_grad():
                return self.policy_net(state).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)
    
    def optimize_model(self):
        if len(self.memory) < BATCH_SIZE:
            return 0.0  # 缓冲区中的样本不足
        
        # 从经验回放缓冲区中采样
        states, actions, rewards, next_states, dones, indices, weights = self.memory.sample(BATCH_SIZE)
        
        # 计算当前Q值
        q_values = self.policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # 使用Double DQN计算下一个状态的Q值
        # 使用策略网络选择动作
        next_actions = self.policy_net(next_states).max(1)[1].unsqueeze(1)
        # 使用目标网络评估动作
        next_q_values = self.target_net(next_states).gather(1, next_actions).squeeze(1)
        # 将终止状态的下一个Q值设为0
        next_q_values = next_q_values * (1 - dones)
        # 计算目标Q值
        target_q_values = rewards + GAMMA * next_q_values
        
        # 计算损失（TD误差）
        td_error = torch.abs(q_values - target_q_values).detach().cpu().numpy()
        loss = F.smooth_l1_loss(q_values, target_q_values, reduction='none') * weights
        loss = loss.mean()
        
        # 优化模型
        self.optimizer.zero_grad()
        loss.backward()
        # 梯度裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 10)
        self.optimizer.step()
        
        # 更新优先级
        self.memory.update_priorities(indices, td_error + 1e-5)
        
        return loss.item()
    
    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def save_model(self, path):
        torch.save({
            'policy_net': self.policy_net.state_dict(),
            'target_net': self.target_net.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'steps_done': self.steps_done,
            'epsilon': self.epsilon
        }, path)
    
    def load_model(self, path):
        checkpoint = torch.load(path)
        self.policy_net.load_state_dict(checkpoint['policy_net'])
        self.target_net.load_state_dict(checkpoint['target_net'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.steps_done = checkpoint['steps_done']
        self.epsilon = checkpoint['epsilon']

## 8. 预处理和状态转换函数

In [44]:
def preprocess_observation(obs):
    # 把堆叠的4帧图像转换为PyTorch的输入格式
    frames = np.array(obs).squeeze(-1)
    tensor = torch.tensor(frames, dtype=torch.float32, device=device).unsqueeze(0)
    return tensor / 255.0  # 归一化

def preprocess_batch_observation(obs):
    # 处理批量观察数据
    frames = np.array(obs).squeeze(-1)
    tensor = torch.tensor(frames, dtype=torch.float32, device=device)
    return tensor / 255.0  # 归一化

## 9. 评估函数

In [45]:
def evaluate(agent, env_id, num_episodes=5, video_prefix="evaluation"):
    # 创建评估环境，包括视频录制
    env = make_env(env_id, 0, capture_video=True, run_name=video_prefix)()  
    episode_rewards = []
    
    for i in range(num_episodes):
        obs, _ = env.reset()
        obs_tensor = preprocess_observation(obs)
        done = False
        total_reward = 0.0
        
        while not done:
            action = agent.select_action(obs_tensor, eval_mode=True).item()
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            
            obs = next_obs
            obs_tensor = preprocess_observation(obs)
        
        episode_rewards.append(total_reward)
    
    env.close()
    return np.mean(episode_rewards), np.std(episode_rewards)

## 10. 训练函数

In [46]:
def train(agent, envs, num_frames):
    # 初始化环境和进度条
    obs = envs.reset()
    obs_tensor = preprocess_batch_observation(obs)
    
    losses = []
    all_rewards = []
    episode_reward = np.zeros(NUM_ENVS)
    episode_length = np.zeros(NUM_ENVS)
    
    progress_bar = tqdm(range(1, num_frames + 1), desc="Training")
    
    # 训练循环
    for frame_idx in progress_bar:
        # 选择动作
        actions = []
        for i in range(NUM_ENVS):
            action = agent.select_action(obs_tensor[i:i+1])
            actions.append(action.item())
        
        # 执行动作
        next_obs, rewards, terminateds, truncateds = envs.step(actions)
        
        # 处理每个环境的数据
        # dones = [t or tr for t, tr in zip(terminateds, truncateds)]

        dones = []
        for t, tr in zip(terminateds, truncateds):
            if isinstance(tr, dict):
                done = t or tr.get("TimeLimit.truncated", False)
            else:
                done = t or tr
            dones.append(done)

        next_obs_tensor = preprocess_batch_observation(next_obs)
        
        # 更新累计奖励和回合长度
        episode_reward += rewards
        episode_length += 1
        
        # 将数据存入经验回放缓冲区
        for i in range(NUM_ENVS):
            agent.memory.push(
                obs_tensor[i:i+1],
                actions[i],
                rewards[i],
                next_obs_tensor[i:i+1],
                float(dones[i])
            )
        
        # 更新观察
        obs = next_obs
        obs_tensor = next_obs_tensor
        
        # 优化模型
        loss = agent.optimize_model()
        losses.append(loss)
        
        # 检查是否有回合结束
        for i, done in enumerate(dones):
            if done:
                # 记录回合结果
                agent.writer.add_scalar("train/episode_reward", episode_reward[i], agent.steps_done)
                agent.writer.add_scalar("train/episode_length", episode_length[i], agent.steps_done)
                all_rewards.append(episode_reward[i])
                
                # 重置回合统计
                episode_reward[i] = 0
                episode_length[i] = 0
        
        # 更新目标网络
        if frame_idx % TARGET_UPDATE == 0:
            agent.update_target_network()
        
        # 记录训练统计信息
        if frame_idx % 1000 == 0:
            mean_reward = np.mean(all_rewards[-100:]) if all_rewards else 0
            mean_loss = np.mean(losses[-100:]) if losses else 0
            agent.writer.add_scalar("train/epsilon", agent.epsilon, frame_idx)
            agent.writer.add_scalar("train/loss", mean_loss, frame_idx)
            agent.writer.add_scalar("train/mean_reward_100", mean_reward, frame_idx)
            
            progress_bar.set_postfix({
                "avg_reward": f"{mean_reward:.2f}",
                "loss": f"{mean_loss:.5f}",
                "epsilon": f"{agent.epsilon:.2f}"
            })
        
        # 保存模型
        if frame_idx % SAVE_INTERVAL == 0:
            save_path = os.path.join(SAVE_PATH, f"model_{frame_idx}.pt")
            agent.save_model(save_path)
            print(f"\n模型已保存到: {save_path}")
        
        # 评估模型
        if frame_idx % EVAL_INTERVAL == 0:
            print("\n开始评估...")
            eval_reward, eval_std = evaluate(
                agent,
                ENV_NAME,
                num_episodes=EVAL_EPISODES,
                video_prefix=f"eval_{frame_idx}"
            )
            agent.writer.add_scalar("eval/mean_reward", eval_reward, frame_idx)
            agent.writer.add_scalar("eval/reward_std", eval_std, frame_idx)
            print(f"评估结果: 平均奖励 = {eval_reward:.2f} ± {eval_std:.2f}")
        
        # 更新代理的步数计数器
        agent.steps_done += 1
    
    # 训练结束，保存最终模型
    final_path = os.path.join(SAVE_PATH, "model_final.pt")
    agent.save_model(final_path)
    print(f"\n最终模型已保存到: {final_path}")

## 11. 主训练流程

In [47]:
# 创建并行环境
envs = make_vec_env(ENV_NAME, NUM_ENVS)

# 获取环境信息
obs_shape = (4, 84, 84)  # 堆叠的4帧，每帧84x84
n_actions = envs.action_space.n

print(f"观察空间形状: {obs_shape}")
print(f"动作空间大小: {n_actions}")

# 创建DQN代理
agent = DQNAgent(obs_shape, n_actions)

# 开始训练
print("开始训练...")
train(agent, envs, NUM_FRAMES)

# 关闭环境
envs.close()

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


观察空间形状: (4, 84, 84)
动作空间大小: 7
开始训练...


Training:   0%|          | 0/1000 [00:00<?, ?it/s]


模型已保存到: ./models/donkey_kong_20250324_170930/model_250.pt

开始评估...


/home/brendan/miniconda3/envs/rl-final/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /mnt/f/workspace/RL-final-proj/Ataris/videos/donkey_kong_20250324_170930 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


评估结果: 平均奖励 = 0.00 ± 0.00

模型已保存到: ./models/donkey_kong_20250324_170930/model_500.pt

开始评估...
评估结果: 平均奖励 = 0.00 ± 0.00

模型已保存到: ./models/donkey_kong_20250324_170930/model_750.pt

开始评估...
评估结果: 平均奖励 = 0.33 ± 0.47

模型已保存到: ./models/donkey_kong_20250324_170930/model_1000.pt

开始评估...
评估结果: 平均奖励 = 0.33 ± 0.47

最终模型已保存到: ./models/donkey_kong_20250324_170930/model_final.pt


## 12. 加载和测试训练好的模型

In [48]:
def play_and_record_video(model_path, env_id, num_episodes=5):
    # 创建环境
    env = make_env(env_id, 0, capture_video=True, run_name="final_test")()  
    
    # 创建代理并加载模型
    obs_shape = (4, 84, 84)  # 堆叠的4帧，每帧84x84
    n_actions = env.action_space.n
    agent = DQNAgent(obs_shape, n_actions)
    agent.load_model(model_path)
    
    # 测试训练好的代理
    rewards = []
    for i in range(num_episodes):
        obs, _ = env.reset()
        obs_tensor = preprocess_observation(obs)
        done = False
        episode_reward = 0
        
        while not done:
            action = agent.select_action(obs_tensor, eval_mode=True).item()
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            
            obs = next_obs
            obs_tensor = preprocess_observation(obs)
            
        rewards.append(episode_reward)
        print(f"Episode {i+1}: Reward = {episode_reward}")
    
    env.close()
    print(f"平均奖励: {np.mean(rewards):.2f} ± {np.std(rewards):.2f}")

In [49]:
# 加载并测试最终模型
model_path = os.path.join(SAVE_PATH, "model_final.pt")
play_and_record_video(model_path, ENV_NAME, num_episodes=5)

Episode 1: Reward = 0.0
Episode 2: Reward = 1.0
Episode 3: Reward = 0.0
Episode 4: Reward = 1.0
Episode 5: Reward = 0.0
平均奖励: 0.40 ± 0.49


## 13. 可视化训练结果

In [50]:
# 使用TensorBoard可视化训练结果
print(f"可以通过以下命令在终端中启动TensorBoard查看训练指标:")
print(f"tensorboard --logdir={LOG_PATH}")

可以通过以下命令在终端中启动TensorBoard查看训练指标:
tensorboard --logdir=./logs/donkey_kong_20250324_170930
